# Reinforcement Learning Techniques in Flappy Bird

Reinforcement learning has been the foundation of breakthroughs in go, chess, and protein folding [^1][^2][^3]. It works by placing neural networks in a simulated environment and rewarding them for good behavior. 

I apply reinforcement learning to the mobile game flappy bird[^4]. Flappy bird is a simple mobile game that uses a discrete state space and a continuous action space, making it a good introduction to reinforcement learning. I start with extremely easy state representations and reward functions, and gradually increase the complexity. Growing the problem size as I succeed at each step provides prerequisite intuition to debug issues that occur as networks become more complex.

[^1]: [Mastering the Game of Go without Human Knowledge](https://discovery.ucl.ac.uk/id/eprint/10045895/1/agz_unformatted_nature.pdf)\
[^2]: [Mastering Chess and Shogi by Planning with a Tree Search](https://arxiv.org/pdf/1712.01815.pdf)\
[^3]: [Highly Accurate Protein Structure Prediction with AlphaFold](https://www.nature.com/articles/s41586-021-03819-2)\
[^4]: [Flappy Bird](https://flappy-bird.co)\
[^5]: This is a continuation of a project that I started two years ago, a final project for a deep learning course. I didn't end up getting it working then, but have been working on it since.

## Introduction to Reinforcement Learning
In reinforcement learning, an agent lives in an environment specified by an event loop. The agent must observe the relationship between its actions and the environment's response. The environment's response consists of two things: (1) a reward signal, and (2) a new state.

``` python
observation = env.get_initial_state()
while True:
    action = agent.act(observation)
    observation, reward = env.step(action)
    agent.learn(observation, reward)
```
Formally, the environment is defined by four things: (1) the state space $S$, (2) the action space $A$, (3) the transition function $P$, and (4) the reward function $R$.

- The state space $S$ is the set of all possible states the agent can observe. 
- The action space $A$ is the set of all possible actions the agent can take.
- The transition function $P(s'|s,a) \rightarrow [0,1]$ is the probability of transitioning from state $s \in S$ to state $s' \in S$ given action $a \in A$. 
- The reward function $R(s,a,s') \rightarrow \mathbb{R}$ is the reward the agent receives for transitioning from state $s$ to state $s'$.

When the agent interacts with the environment, its interaction can be described as a sequence of states, actions, and rewards:

$$
(s_0, a_0, r_0, s_1, a_1, r_1, \cdots)
$$

[^6]: [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/RLbook2020.pdf)

## Flappy Bird Environment
I used the flappy bird gymnasium package to create the environment. 
The Flappy Bird environment is characterized by a state space, an action space, and a reward function. 
1. The state space $S \in \mathbb{R}^{12}$ is a vector of 12 numbers, representing various distances, velocities, and angles related to the bird and the pipes. See [^8] for complete details. 
2. The action space $A = \{0, 1\}$ allows the agent to either "do nothing" or "flap". 
3. The transition probability function $P$ is unknown, and the agent must learn it. If $P$ was known prior, then we could compute the optimal policy trivially with the Bellman Equation [^9]. 
4. The reward function $R$ provides feedback based on the agent's state transitions, rewarding survival, successful pipe navigation, and penalizing death. The reward function is defined as:

 $R(s, a) = \begin{cases} +0.1 & \text{if the agent is alive} \\ +1.0 & \text{if the agent passes through a pipe} \\ -1.0 & \text{if the agent dies} \end{cases}$


[^7]: [Flappy Bird Gymnasium](https://github.com/Kautenja/flappy-bird-gymnasium)\
[^8]: [Flappy Bird State Vector](https://github.com/markub3327/flappy-bird-gymnasium)\
[^9]: Reinforcement learning: an introduction Section

## Sanity Check for the Flappy Bird Environment

Before exploring reinforcement learning techniques, it is important to sanity check the flappy bird environment. Sanity testing the environment will make us more knowledgeable when debugging future issues.

In [18]:
import flappy_bird_gymnasium
import gymnasium
import numpy as np
from tqdm import tqdm
from agent import Agent
import os

In [ ]:
import gymnasium
import cv2
import numpy as np
from IPython.display import Video

# Create the environment
env = gymnasium.make("FlappyBird-v0", 
                     render_mode="rgb_array",
                     use_lidar=False,
                     normalize_obs=True)

fourcc = cv2.VideoWriter_fourcc(*'avc1')

video_name = 'sanity_check0.mp4'
need_to_create_video = not os.path.exists(video_name)
if need_to_create_video:
    out = cv2.VideoWriter(video_name, fourcc, 30.0, (288, 512))

observations = np.zeros((0, 12))
env.reset()
while True:
    obs, reward, terminal, _, _ = env.step(env.action_space.sample())
    observations = np.vstack((observations, obs))
    image = env.render()
    
    # Convert the image from RGB to BGR
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Write the frame to the video file
    if need_to_create_video:
        out.write(image_bgr)
    
    if terminal:
        break    

# Release resources
env.close()
out.release()

# Display the converted video in the notebook
Video(video_name, embed=True)
print(observations.shape)

This sanity check confirms that the state space is 12-dimensional, and that the environment renders as expected.

## Reward
The agent's goal is to maximize its return, the cumulative reward over time. The expected return is
$$\sum_{k=0}^\infty\gamma^kR_{t+k+1}$$
where $\gamma$ is the discount rate, a penalty applied to future rewards because they are less certain.

## Q-Learning
In Q-Learning approximates a function $Q(s,a) \rightarrow \mathbb{R}, \text{where }s \in \mathcal{S}, a \in \mathcal{A}$ to achieve the highest return in a trajectory. 
$$
Q(s,a) = Q(s,a) + \alpha[R_t + \gamma(\argmax_{a}(Q(s',a)) - Q(s,a))]
$$

To start, I handcrafted a far simpler version of the state space in flappy bird where:
 $$State = \begin{cases} 1 & \text{jumping is optimal} \\
  0 & \text{not jumping is optimal} \\
\end{cases}$$

This handcrafted reward system is not perfect, but results in significantly better performance than the other case. An optimal agent would perform similarly, but not identically to this model. The handcrafted representation makes evaluating whether future models simpler by comparing the future models policy to the handcrafted policy. Because Q(s,a) naturally involves some exploration, exploration must be turned off before evaluating the quality of the function.
### Untrained Agent

In [2]:
import flappy_bird_gymnasium
import gymnasium
import numpy as np
from tqdm import tqdm
from agent import Agent

env = gymnasium.make("FlappyBird-v0", 
                    # render_mode="human",
                    use_lidar=False,
                    normalize_obs=True)
agent = Agent()

for i in range(int(1e6)):
    current_obs, _ = env.reset()
    if (i+1) % 100 == 0:
        print(f"Episode {i+1}")
        print(agent.qtable)
    act = agent.demo_act if (i+1) % 1000 == 0 else agent.act
    act = agent.demo_act
    while True:
        state = agent.discretize_state(current_obs)

        action = agent.act(state)

        next_obs, reward, terminated, truncated, info = env.step(action)
        next_state = agent.discretize_state(next_obs)
        agent.learn(state, action, reward, next_state, terminated)
        
        if terminated:
            break 
        current_obs = next_obs
env.close()


Episode 100
[[0.00179114 0.00010783]
 [0.00182666 0.00011814]]
Episode 200
[[0.00301974 0.00018332]
 [0.00391069 0.00022874]]
Episode 300
[[0.00450351 0.00027342]
 [0.00576536 0.00034599]]
Episode 400
[[0.00576136 0.00033854]
 [0.00781108 0.00048725]]
Episode 500
[[0.00726559 0.00040241]
 [0.00968072 0.00060255]]
Episode 600
[[0.00862435 0.00047898]
 [0.01168487 0.00073105]]
Episode 700
[[0.00978769 0.00054696]
 [0.01365629 0.00085106]]
Episode 800
[[0.01128677 0.00062552]
 [0.01556837 0.00102541]]
Episode 900
[[0.01260919 0.00068268]
 [0.01749984 0.00117636]]
Episode 1000
[[0.01420174 0.00076953]
 [0.01936488 0.00134533]]
Episode 1100
[[0.01551394 0.00079117]
 [0.02135159 0.00151054]]
Episode 1200
[[0.01710656 0.00088392]
 [0.02318762 0.00166535]]
Episode 1300
[[0.01835271 0.00096194]
 [0.02531136 0.001835  ]]
Episode 1400
[[0.02011271 0.00102741]
 [0.02699202 0.00202197]]
Episode 1500
[[0.02180144 0.00114869]
 [0.02871099 0.00219662]]
Episode 1600
[[0.02340928 0.00123996]
 [0.0304392

KeyboardInterrupt: 



### Agent after 1.5 million iterations

In this game with simplified state, the agent's learning process was extremely inefficient. It required 1.5 million iterations to learn the optimal jumping policy, which is surprising because the game has two discrete states and two discrete actions. The reason the agent took so long to converge was because it could only learn by measuring the payoff after an $\epsilon$-greedy exploration. This work can be extended by measuring how well this does for different values of $\epsilon$ and how policy-based methods would do a better job of learning the optimal policy.
